# Simplified Features

Within this section I'll take the data generated in the previous section and simplify the features used by the model. This section is important as I'd like to create a more user friendly model, at the expense of model accuracy. Simplifying, for instance, the soil types from probability of soil X,Y,Z to is_soil 'Clay'? will be beneficial to make the deployed model more useable.

### Library Imports

In [1]:
import numpy as np
from numpy.linalg import inv

import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import seaborn as sns
import shap

import plotly.express as px
import pickle
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.graph_objects as go


import pycaret
from pycaret.regression import *
from pycaret.regression import RegressionExperiment
from pycaret.regression import get_config
from pycaret.regression import save_model, load_model

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_or

In [2]:
pd.set_option('display.max_columns', None)

### Data Import

In [3]:
with open('pickles/data_adj.pkl', 'rb') as f:
    data_adj = pickle.load(f)
with open('pickles/data_adj_unseen.pkl', 'rb') as f:
    data_adj_unseen = pickle.load(f)
with open('pickles/df_soil.pkl', 'rb') as f:
    df_soil = pickle.load(f)
with open('pickles/df_adj.pkl', 'rb') as f:
    df_adj = pickle.load(f)
with open('pickles/df_streamlit.pkl', 'rb') as f:
    df_streamlit = pickle.load(f)   
with open('pickles/data_subset.pkl', 'rb') as f:
    data_subset = pickle.load(f)
with open('pickles/data_subset.pkl', 'rb') as f:
    data_subset = pickle.load(f)
with open('pickles/combined_metrics.pkl', 'rb') as f:
    combined_metrics = pickle.load(f)    

In [4]:
with open('pickles/df_streamlit.pkl', 'rb') as f:
    df_streamlit = pickle.load(f)   

In [5]:
df_user = pd.concat([data_adj,data_adj_unseen],ignore_index=True)

In [6]:
df_adj.columns

Index(['country', 'city', 'start_year', 'end_year', 'rr?', 'length',
       'tunnel_per', 'tunnel', 'elevated', 'at_grade', 'stations', 'max_speed',
       'track_gauge', 'overhead?', 'cost', 'currency', 'year', 'ppp_rate',
       'cost_real', 'cost_km', 'c_length', 'c_tunnel', 'anglo',
       'inflation_index', 'cost_real_2021', 'cost_km_2021', 'duration',
       'country_pop_den', 'region', 'sub_region', 'area_km', 'lat', 'lng',
       'population', 'time_diff', 'calculated_population', 'city_density',
       'per_below_line', 'reporting_gdp', 'wrb_class_name', 'wrb_class_value',
       'prob_Acrisols', 'prob_Albeluvisols', 'prob_Alisols', 'prob_Andosols',
       'prob_Arenosols', 'prob_Calcisols', 'prob_Cambisols', 'prob_Chernozems',
       'prob_Cryosols', 'prob_Durisols', 'prob_Ferralsols', 'prob_Fluvisols',
       'prob_Gleysols', 'prob_Gypsisols', 'prob_Histosols', 'prob_Kastanozems',
       'prob_Leptosols', 'prob_Lixisols', 'prob_Luvisols', 'prob_Nitisols',
       'prob_Phaeoz

In [ ]:
df_place = df_adj[['country','city','length','stations','track_gauge','year']]

In [ ]:
df_place

In [ ]:
df_user = pd.merge(df_user,df_place,how ='left',on=['stations','length','track_gauge','year'])

In [ ]:
df_user.head(5)

## Feature Simplification

### Soil Types

I'd like to simplify the World Reference Base for Soil Resources'soil class name' nomenclature used in the dataset by reducing each class name into a type of soil. 

Instead of AC Acrisol (low-activity clays, exchangeable Al > exchangeable base cations), I'd like to refine this to 'clays'.

In [ ]:
df_soil_type = df_soil[['wrb_class_name','wrb_class_value']].drop_duplicates()
df_soil_type.head(10)

In [ ]:
df_user = pd.merge(df_user,df_soil_type,on=['wrb_class_value'],how='left')

In [ ]:
soil_probs = [
    "prob_Acrisols", "prob_Albeluvisols", "prob_Alisols", "prob_Andosols",
    "prob_Arenosols", "prob_Calcisols", "prob_Cambisols", "prob_Chernozems",
    "prob_Cryosols", "prob_Durisols", "prob_Ferralsols", "prob_Fluvisols",
    "prob_Gleysols", "prob_Gypsisols", "prob_Histosols", "prob_Kastanozems",
    "prob_Leptosols", "prob_Lixisols", "prob_Luvisols", "prob_Nitisols",
    "prob_Phaeozems", "prob_Planosols", "prob_Plinthosols", "prob_Podzols",
    "prob_Regosols", "prob_Solonchaks", "prob_Solonetz", "prob_Stagnosols",
    "prob_Umbrisols", "prob_Vertisols"
]

In [ ]:
df_user.drop(columns=soil_probs,inplace=True)

In [ ]:
df_user

In [ ]:
df_user['wrb_class_name'].value_counts()

In [ ]:
def map_to_simplified_class(wrb_class_name):
    organic = ['Histosols']
    human_altered = ['Anthrosols', 'Technosols']
    cold_climates = ['Cryosols', 'Leptosols']
    mineral_rich = ['Andosols', 'Podzols', 'Plinthosols', 'Ferralsols','Gleysols']
    high_altitude_or_shallow = ['Stagnosols', 'Nitisols', 'Regosols']
    fertile = ['Chernozems', 'Kastanozems', 'Phaeozems', 'Umbrisols', 'Cambisols']
    saline_arid = ['Arenosols','Durisols', 'Gypsisols', 'Calcisols', 'Planosols','Solonchaks','Solonetzs']
    clay_dominant = ['Retisols', 'Acrisols', 'Lixisols', 'Alisols', 'Luvisols','Albeluvisols','Vertisols']
    river_valleys = ['Fluvisols']
    
    if wrb_class_name in organic:
        return 'Organic (Bogs & Peats)'
    elif wrb_class_name in human_altered:
        return 'Human-altered (Urban, Cut/Fill, Artifacts)'
    elif wrb_class_name in cold_climates:
        return 'Cold Climates (Permafrost, Rock Outcrops)'
    elif wrb_class_name in mineral_rich:
        return 'Environment Dependent (Wetlands, Volcanic Ash, Mineral Rich)'
    elif wrb_class_name in high_altitude_or_shallow:
        return 'High Altitude/Wet (Mountain, Swampy)'
    elif wrb_class_name in fertile:
        return 'Fertile/Agricultural (Grasslands, Food Bearing, Pasture)'
    elif wrb_class_name in saline_arid:
        return 'Saline/Arid (Desert Soils, High Salt Content)'
    elif wrb_class_name in clay_dominant:
        return 'Clay Dominant'
    elif wrb_class_name in river_valleys:
        return 'River Valleys/Deltas (River Sediments)'
    else:
        return 'Others'

df_user['soil_type'] = df_user['wrb_class_name'].apply(map_to_simplified_class)

In [ ]:
print(df_user['soil_type'].value_counts())

In [ ]:
df_user.drop(columns= ['wrb_class_name','wrb_class_value'],inplace=True)

In [ ]:
df_user.head(5)

### Track Gauge

In [ ]:
df_user['track_gauge'].drop_duplicates()

In [ ]:
def gauge_mapping(value):
    if value == 0:
        return 'monorail'
    elif value == 1435:
        return 'standard'
    elif value < 1435:
        return 'narrow'
    else:
        return 'wide'

df_user['gauge_width'] = df_user['track_gauge'].apply(gauge_mapping)

In [ ]:
column_to_plot = "gauge_width"

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

### City Population

Instead of using population numbers, I'll redefine the population as 'city size' where the user will select from a range of qualitative options instead of specific populations.

In [ ]:
df_user.sort_values(by='calculated_population',ascending=False)

In [ ]:
column_to_plot = "calculated_population"

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

In [ ]:
df_user.sort_values(by='calculated_population',ascending=False)

In [ ]:
def pop_mapping(value):
    if value < 250000:
        return '<250k (small)'
    elif (value >=250000) &(value<500000):
        return '250k-500k (medium-small)'
    elif (value >=500000) & (value<1000000):
        return '500k-1M (medium)'
    elif (value >=1000000) &(value<2000000):
        return '1M-2M (medium-large)'
    elif (value >=2000000) &(value<5000000):
        return '2M-5M (large)'
    elif (value >=5000000) &(value<10000000):
        return '5M-10M (very large)'
    elif (value >=10000000) &(value<15000000):
        return '10M-15M (small Metropolis)'
    else:
        return '>15M (Metropolis)'

df_user['city_size'] = df_user['calculated_population'].apply(pop_mapping)

In [ ]:
column_to_plot = "city_size" 

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)
plt.show()

#### Train Type

Within the dataset, I make a distinction between different types of trains however I don't explicitely dicipher between different types of metros, i.e. subways and at-grade trams. I'll do that below using the tunnel_per feature and the max speed feature.

In [ ]:
def train_type_mapping(row):
    max_speed = row['max_speed']
    tunnel_per = row['tunnel_per']
    track_gauge = row['track_gauge']
    elevated = row['elevated']
    at_grade = row['at_grade']
    length = row['length']

    if (max_speed <= 60) & (tunnel_per == 0):
        return 'tram'
    elif (tunnel_per >.8):
        return 'subway'
    elif (track_gauge ==0):
        return 'monorail'
    elif (tunnel_per < 1) & (elevated > .8*length):
        return 'elevated light rail'
    else:
        return 'light rail'

df_user['train_type'] = df_user.apply(train_type_mapping,axis=1)

In [ ]:
column_to_plot = "train_type"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

### Country Wealth

In this feature, I'll combine the population and the GDP to create a GDP per capita. I'll then classify that GDP per capita into distinct categories, as defined by the World Economic Forum.

In [ ]:
df_user['gdp_cap'] = (df_user['reporting_gdp']*1000000)/df_user['calculated_population']

In [ ]:
def wealth_mapping(value):
    if (value <= 1045):
        return 'low-income'
    elif (value > 1045) & (value < 4095):
        return 'lower-middle income'
    elif (value > 4096) & (value < 12695):
        return 'upper-middle income'
    else:
        return 'high-income'

df_user['country_income_class'] = df_user['gdp_cap'].apply(wealth_mapping)

In [ ]:
column_to_plot = "country_income_class"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

### Elevation

In [ ]:
column_to_plot = "elevation"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=True)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

In [ ]:
def elevation_mapping(value):
    if (value <= 200):
        return 'Coastal'
    elif (value > 200) & (value <= 1000):
        return 'Mid-land'
    elif (value > 1000) & (value <= 2500):
        return 'High-land'
    elif (value > 2500) & (value <= 3500):
        return 'Mountainous'
    else:
        return 'High Altitude'

df_user['elevation_class'] = df_user['elevation'].apply(elevation_mapping)

In [ ]:
column_to_plot = "elevation_class"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

### Precipitation

In [ ]:
column_to_plot = "prcp"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=True)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

In [ ]:
def precipitation_mapping(value):
    if value <= 10:
        return 'Very-Arid'
    elif (value > 10) & (value <= 20):
        return 'Arid'
    elif (value > 20) & (value <= 40):
        return 'Semi-Arid'
    elif (value > 40) & (value <= 60):
        return 'Low-Moderate-Rainfall'
    elif (value > 60) & (value <= 100):
        return 'Moderate-Rainfall'
    elif (value > 100) & (value <= 150):
        return 'High-Moderate-Rainfall'
    elif (value > 150) & (value <= 200):
        return 'High-Rainfall'
    elif (value > 200) & (value <= 250):
        return 'Very-High-Rainfall'
    else:
        return 'Extreme-Rainfall/Tropical'

df_user['precipitation_type'] = df_user['prcp'].apply(precipitation_mapping)

In [ ]:
column_to_plot = "precipitation_type"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=90)
plt.show()

### Temperature

In [ ]:
def temperature_mapping(row):
    avg_temp = row['tavg']
    t_min = row['tmin']
    t_max = row['tmax']

    if avg_temp <= 5 and t_min >= -30 and t_max <= 10:
        return 'Very Cold'
    elif avg_temp <= 15 and t_min >= -20 and t_max <= 20:
        return 'Cold/Cool'
    elif avg_temp <= 25 and t_min >= -10 and t_max <= 30:
        return 'Mild/Moderate'
    elif avg_temp <= 33 and t_min >= 0 and t_max <= 33:
        return 'Warm/Hot'
    else:
        return 'Very Hot or Extreme Heat'

df_user['temperature_category'] = df_user.apply(temperature_mapping, axis=1)

In [ ]:
column_to_plot = "temperature_category"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

In [ ]:
print(df_user['temperature_category'].value_counts())

### Affordability

In [ ]:
df_user['affordability_index'].describe()

In [ ]:
def affordability_mapping(value):
    if value == 0:
        return 'Very Unaffordable'
    elif value <= 1:
        return 'Unaffordable'
    elif value <= 5:
        return 'Moderately Affordable'
    elif value < 7:
        return 'Affordable'
    else:
        return 'Highly Affordable'

df_user['affordability'] = df_user['affordability_index'].apply(affordability_mapping)

In [ ]:
column_to_plot = "affordability"

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.show()

### Union Prevalence

In [ ]:
sns.ecdfplot(data=df_user, x="union_density")
plt.show()

In [ ]:
df_user['union_density'].describe()

In [ ]:
def union_density_mapping(value):
    if value <= 13:
        return 'Very Few/No Labor Unions'
    elif value <= 14:
        return 'Some Labor Unions'
    elif value <= 30:
        return 'Many Labor Unions'
    else:
        return 'Most People are Part of a Labor Union'

df_user['union_prevalence'] = df_user['union_density'].apply(union_density_mapping)

In [ ]:
column_to_plot = "union_prevalence"  
# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)

plt.show()

### Poverty Rates

In [ ]:
df_user.head(5)

In [ ]:
column_to_plot = "per_below_line"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

In [ ]:
df_user['per_below_line'].describe()

In [ ]:
def poverty_mapping(value):
    if value <= 3:
        return 'Very Little Poverty'
    elif value <= 15:
        return 'Little Poverty'
    elif value <= 30:
        return 'Some Poverty'
    elif value <= 50:
        return 'Very Impoverished'
    else:
        return 'Mostly Impoverished'

df_user['poverty_rate'] = df_user['per_below_line'].apply(poverty_mapping)

In [ ]:
column_to_plot = "poverty_rate"

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

### City & Country Density

Previously I had create a feature called 'urban', which was just a binary gate that indicated whether the city density was beyond a certain threshold. I'll expand that idea further here.

In [ ]:
column_to_plot = "city_density"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=True)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

In [ ]:
df_user['city_density'].describe()

In [ ]:
def city_density_mapping(value):
    if value <= 1523:
        return 'Not Dense'
    elif value <= 2779:
        return 'Somewhat Dense'
    elif value <= 5607:
        return 'Dense'
    else:
        return 'Very Dense'

df_user['city_density_type'] = df_user['city_density'].apply(city_density_mapping)

In [ ]:
column_to_plot = "city_density_type"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

In [ ]:
column_to_plot = "country_pop_den"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=True)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

In [ ]:
df_user['country_pop_den'].describe()

In [ ]:
def country_density_mapping(value):
    if value <= 114:
        return 'Not Dense'
    elif value <= 125:
        return 'Marginally Dense'
    elif value <= 150:
        return 'Dense'
    else:
        return 'Very Dense'

df_user['country_density_type'] = df_user['country_pop_den'].apply(country_density_mapping)

In [ ]:
column_to_plot = "country_density_type"  # Replace with the name of the column you want to plot

# Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_user[column_to_plot], kde=False)
plt.title(f'Histogram of {column_to_plot}')
plt.xticks(rotation=45)  # This line tilts the x labels by 45 degrees

plt.show()

### Anglo?

Currently the 'Anglo' category is a binary gate where a '1' indicates that the country is anglo and '0' indicates not. I want this to be more user friendly and so I'll convert these to 'yes' or 'no'.

In [ ]:
df_user['anglo'] = np.where(df_user['anglo'] == 1,'yes','no')

In [ ]:
df_user.rename(columns = {"anglo":"anglo?"},inplace= True)

In [ ]:
user_drop = ['country','city','rr?','tunnel_per',
             'max_speed',
             'track_gauge',
             'overhead?','year','country_pop_den',
             'area_km','population','calculated_population',
             'city_density','per_below_line','reporting_gdp',
             'elevation','tavg','tmin','tmax','prcp','gdp_cap',
             'land_cost_year','price_income_ratio','mortgage_perc_income',
             'affordability_index','urban?','rental_yield',
             'price_rent_ratio','union_density','stations_per_km']

In [ ]:
df_user

In [ ]:
df_user.drop(columns = user_drop,inplace=True)

In [ ]:
df_user.head(10)

### Simplified Model

In [ ]:
data = df_user.sample(frac=0.8, random_state=786)
data_unseen = df_user.drop(data.index)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
cont_feats = ['length','tunnel','elevated','at_grade','duration',]
cat_feats =  ["region", "sub_region", "soil_type", "gauge_width", 
    "city_size", "train_type", 
    "country_income_class", "elevation_class", "precipitation_type", 
    "temperature_category", "affordability", "union_prevalence",
    "poverty_rate", "city_density_type", "country_density_type","anglo?"
]

In [ ]:
s = setup(data,
          target = 'cost_real_2021',
          categorical_features= cat_feats,
          numeric_features= cont_feats,
          normalize = True,
          normalize_method = 'zscore',
          verbose = False,
          feature_selection = False,
          low_variance_threshold = 0.1,
          pca = False, 
          pca_components = 30,
          remove_multicollinearity = False,
          multicollinearity_threshold = 0.3,
          memory= False,
          system_log= False
         )

In [ ]:
compare_models(exclude = ['lar','lr','llar'])

#### Individual Models

###### CatBoost

In [ ]:
cat = create_model('catboost')
param_grid_cat = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate.
    'depth': [3, 4, 5, 6, 7, 8, 10],  # Depth limit.
    'l2_leaf_reg': [1, 3, 5, 7, 9],  # Coefficient at the L2 regularization term of weights for leaf value.
    'bagging_temperature': [0.2, 0.5, 0.8, 1.0],  # The settings of the Bayesian bootstrap during training.
    'border_count': [32, 64, 128, 255],  # The number of splits for numerical features.
    'iterations': [50, 100, 200, 300],  # The maximum number of trees to be trained.
    'loss_function': ['RMSE', 'Logloss', 'MultiClass'],  # Objective function.
    'random_strength': [0.5, 1, 2, 3],  # The amount of randomness to use for scoring splits.
    'boosting_type': ['Ordered', 'Plain'],  # Boosting type implementation.
    'subsample': [0.5, 0.7, 0.9, 1.0],  # Sample rate for bagging.
    'rsm': [0.5, 0.7, 0.9, 1.0],  # (Equivalent to colsample_bytree in XGBoost) The percentage of features to use at each split selection.
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']  # Control the way new splits are added to the tree.
}
cat_tuned = tune_model(
    cat,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_cat,
    choose_better = False
)
predict_model(cat_tuned, data=data_unseen)
cat_bagged = cat_tuned

###### ExtraTrees

In [ ]:
et = create_model('et')
param_grid_et = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],  # Set bootstrap option. Default for ExtraTrees is False, but you can experiment with True.
    'max_features': ['auto', 'sqrt', 'log2']
}
et_tuned = tune_model(
    et,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_et,
    choose_better = False
)
predictions = predict_model(et_tuned, data=data_unseen)
et_bagged = ensemble_model(et_tuned,method = 'Bagging',fold = 10,n_estimators = 100, choose_better = True)

###### LightGBM

In [ ]:
light = create_model('lightgbm')
param_grid_light = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 62, 127, 255],
    'max_depth': [3, 4, 5, 6, 7, 8, 10, -1],  # -1 means no limit.
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    'n_estimators': [50, 100, 200, 300],
    'objective': ['regression', 'binary', 'multiclass'],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0, 0.5, 1],
    'boosting_type': ['gbdt', 'dart', 'goss']
}
light_tuned = tune_model(
    light,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_light,
    choose_better = False
)
predictions = predict_model(light_tuned, data=data_unseen)
light_bagged = ensemble_model(light_tuned,method = 'Bagging',fold = 10,n_estimators = 100, choose_better = True)

### Blended Model

In [ ]:
blended_user = blend_models([cat_bagged,et])

In [ ]:
predictions = predict_model(blended_user, data=data_unseen)

In [ ]:
plot_model(blended_user, plot = 'residuals_interactive')

### Cooks Distance

In [ ]:
X_encoded = pd.get_dummies(df_user.drop(columns=['cost_real_2021', 'region', 'sub_region']), drop_first=True)
X_encoded['Intercept'] = 1
X_encoded = X_encoded[['Intercept'] + [col for col in X_encoded if col != 'Intercept']]

In [ ]:
H = X_encoded.values @ np.linalg.inv(X_encoded.values.T @ X_encoded.values) @ X_encoded.values.T
leverage = np.diag(H)

In [ ]:
df_user['leverage'] = leverage
df_user.sort_values(by='leverage',ascending=False)

In [ ]:
predictions_seen = predict_model(blended_user, data=data)

In [ ]:
df_user['residuals'] = predictions_seen['cost_real_2021'] - predictions_seen['prediction_label']
mean_res = np.mean(df_user['residuals'])
std_res = np.std(df_user['residuals'])

df_user['standardized_residuals'] = (df_user['residuals'] - mean_res) / std_res

In [ ]:
p = X_encoded.shape[1]
mse = np.mean(df_user['residuals'] ** 2)
df_user['cooks_distance'] = (df_user['residuals'] ** 2 / (p * mse)) * (df_user['leverage'] / (1 - df_user['leverage']) ** 2)
df_user.sort_values(by='cooks_distance',ascending=False)

In [ ]:
n = df_user.shape[0]
threshold = 4/n
threshold

In [ ]:
cooks_outliers = df_user[df_user['cooks_distance']>threshold].sort_values(by='cooks_distance',ascending=False)
cooks_outliers

In [ ]:
cooks_outliers = df_user[df_user['cooks_distance']>threshold].sort_values(by='cooks_distance',ascending=False).index
len(cooks_outliers)

In [ ]:
df_user.drop(index = cooks_outliers,inplace = True)

In [ ]:
df_user.sort_values(by='standardized_residuals',ascending=False,key=abs)

In [ ]:
high_res = df_user[abs(df_user['standardized_residuals'])> 2].index.tolist()

In [ ]:
df_user.drop(index = high_res,inplace = True)

In [ ]:
df_user

### Re-Run Model

In [ ]:
df_user.drop(columns =['cooks_distance','standardized_residuals','residuals','leverage'],inplace=True)

In [ ]:
data = df_user.sample(frac=0.8, random_state=786)
data_unseen = df_user.drop(data.index)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

In [ ]:
data.head(5)

In [ ]:
cont_feats = ['length','tunnel','elevated','at_grade','duration',]
cat_feats =  ["region", "sub_region", "soil_type", "gauge_width", 
    "city_size", "train_type", 
    "country_income_class", "elevation_class", "precipitation_type", 
    "temperature_category", "affordability", "union_prevalence",
    "poverty_rate", "city_density_type", "country_density_type","anglo?"]

In [ ]:
s = setup(data,
          target = 'cost_real_2021',
          categorical_features= cat_feats,
          numeric_features= cont_feats,
          normalize = True,
          normalize_method = 'zscore',
          verbose = False,
          feature_selection = False,
          low_variance_threshold = 0.1,
          pca = False, 
          pca_components = 30,
          remove_multicollinearity = False,
          multicollinearity_threshold = 0.3,
          imputation_type= 'iterative',
          categorical_imputation='mode',
          numeric_imputation = 'mode',
          memory= False,
          system_log= False
         )

In [ ]:
compare_models(exclude = ['lar','knn','dt','ada'])

In [ ]:
cat = create_model('catboost')
param_grid_cat = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate.
    'depth': [3, 4, 5, 6, 7, 8, 10],  # Depth limit.
    'l2_leaf_reg': [1, 3, 5, 7, 9],  # Coefficient at the L2 regularization term of weights for leaf value.
    'bagging_temperature': [0.2, 0.5, 0.8, 1.0],  # The settings of the Bayesian bootstrap during training.
    'border_count': [32, 64, 128, 255],  # The number of splits for numerical features.
    'iterations': [50, 100, 200, 300],  # The maximum number of trees to be trained.
    'loss_function': ['RMSE', 'Logloss', 'MultiClass'],  # Objective function.
    'random_strength': [0.5, 1, 2, 3],  # The amount of randomness to use for scoring splits.
    'boosting_type': ['Ordered', 'Plain'],  # Boosting type implementation.
    'subsample': [0.5, 0.7, 0.9, 1.0],  # Sample rate for bagging.
    'rsm': [0.5, 0.7, 0.9, 1.0],  # (Equivalent to colsample_bytree in XGBoost) The percentage of features to use at each split selection.
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']  # Control the way new splits are added to the tree.
}
cat_tuned = tune_model(
    cat,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_cat,
    choose_better = False
)
predict_model(cat_tuned, data=data_unseen)

In [ ]:
et = create_model('et')
param_grid_et = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],  # Set bootstrap option. Default for ExtraTrees is False, but you can experiment with True.
    'max_features': ['auto', 'sqrt', 'log2']
}
et_tuned = tune_model(
    et,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_et,
    choose_better = False
)
predictions = predict_model(et_tuned, data=data_unseen)
et_bagged = ensemble_model(et_tuned,method = 'Bagging',fold = 10,n_estimators = 100, choose_better = True)

#### LightGBM

In [ ]:
light = create_model('lightgbm')
param_grid_light = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 62, 127, 255],
    'max_depth': [3, 4, 5, 6, 7, 8, 10, -1],  # -1 means no limit.
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    'n_estimators': [50, 100, 200, 300],
    'objective': ['regression', 'binary', 'multiclass'],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0, 0.5, 1],
    'boosting_type': ['gbdt', 'dart', 'goss']
}
light_tuned = tune_model(
    light,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_light,
    choose_better = False
)
predictions = predict_model(light_tuned, data=data_unseen)
light_bagged = ensemble_model(light_tuned,method = 'Bagging',fold = 10,n_estimators = 100, choose_better = True)

### Blended Model

In [ ]:
blended_user_reduced = blend_models([cat,light_bagged,et])

In [ ]:
predictions_blended = predict_model(blended_user_reduced, data=data_unseen)

#### Tuning Blended Model

In [ ]:
blended_user_tuned = blend_models([cat,light_bagged,et],weights =[.7,.15,.15])

In [ ]:
predictions_tuned = predict_model(blended_user_tuned, data=data_unseen)

### Finalized Model

In [ ]:
finalized_user_model = finalize_model(blended_user_tuned)

In [ ]:
predictions_user = predict_model(finalized_user_model, data=data_unseen)

In [ ]:
interpret_model(blended_user_tuned, plot = 'msa')

In [ ]:
interpret_model(light, plot = 'pfi')

### SHAP Plot (in Plotly)

In [ ]:
data_encoded = pd.get_dummies(data, columns=cat_feats)
data_unseen_encoded = pd.get_dummies(data_unseen, columns=cat_feats)
data_encoded

In [ ]:
missing_cols = set(data_encoded.columns) - set(data_unseen_encoded.columns)
for c in missing_cols:
    data_unseen_encoded[c] = 0

data_unseen_encoded = data_unseen_encoded[data_encoded.columns]

#### Model for SHAP

In [ ]:
s = setup(data_encoded,
          target = 'cost_real_2021',
          normalize = False,
          normalize_method = 'zscore',
          verbose = False,
          memory= False,
          system_log= False
         )

In [ ]:
light_encoded = create_model('lightgbm')
param_grid_light = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 62, 127, 255],
    'max_depth': [3, 4, 5, 6, 7, 8, 10, -1],  # -1 means no limit.
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    'n_estimators': [50, 100, 200, 300],
    'objective': ['regression', 'binary', 'multiclass'],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0, 0.5, 1],
    'boosting_type': ['gbdt', 'dart', 'goss']
}
light_tuned_encoded = tune_model(
    light,
    fold = 10,
    n_iter = 50,
    custom_grid = param_grid_light,
    choose_better = False
)
predictions = predict_model(light_tuned_encoded, data=data_unseen_encoded)

In [ ]:
X_transformed = get_config('X')
explainer = shap.TreeExplainer(light_tuned_encoded)
shap_values = explainer.shap_values(X_transformed)
y_pred = predictions['prediction_label'].values

In [ ]:
df_plot = pd.DataFrame(shap_values, columns=X_transformed.columns)
df_plot_melted = df_plot.melt(var_name="Feature", value_name="SHAP")

In [ ]:
repeat_times = df_plot_melted.shape[0] // y_pred.shape[0]
remainder = df_plot_melted.shape[0] % y_pred.shape[0]
new_tiled_array = np.concatenate([np.tile(y_pred, repeat_times), y_pred[:remainder]])
df_plot_melted["predictions"] = new_tiled_array

# Aggregate and sort SHAP values
agg_shap_total = df_plot_melted.groupby('Feature').agg({'SHAP': 'sum'}).reset_index()
agg_shap_total = agg_shap_total.sort_values(by='SHAP', ascending=False)

# Get the top 10 features based on total SHAP
top_features = agg_shap_total['Feature'].head(10).tolist()
df_plot_melted = df_plot_melted[df_plot_melted['Feature'].isin(top_features)]

# Sort by SHAP value and calculate size
df_plot_melted = df_plot_melted.sort_values(by="SHAP", key=abs, ascending=True)
df_plot_melted["size"] = df_plot_melted["SHAP"].abs()
df_plot_melted["difference"] = abs(df_plot_melted["predictions"]+df_plot_melted["SHAP"])

shap_range = df_plot_melted.groupby('Feature')['SHAP'].agg(lambda x: x.max() - x.min()).reset_index()
shap_range.columns = ['Feature', 'SHAP_Range']

# Merge this with the main dataframe
df_plot_melted = df_plot_melted.merge(shap_range, on='Feature')
df_plot_melted['abs_avg'] = np.mean(abs(df_plot_melted['SHAP']))
# Sort by SHAP range
df_plot_melted = df_plot_melted.sort_values(by="abs_avg", ascending=True)

In [ ]:
fig = px.scatter(df_plot_melted, x='SHAP', y='Feature', color='SHAP', size='difference', 
                 color_continuous_scale='plasma', height=800, width=800)

fig.update_layout(
    xaxis=dict(showgrid=True, gridcolor='WhiteSmoke', zerolinecolor='Gainsboro'),
    yaxis=dict(showgrid=True, gridcolor='WhiteSmoke', zerolinecolor='Gainsboro'),
    plot_bgcolor='white',
    boxgap=1
)
fig.update_traces(marker=dict(line=dict(width=0)))

# Save and show plot
fig.write_html('plots/plotly_beeswarm_top10.html')
fig.show()

### Importances Plot

In [ ]:
importances = light_tuned_encoded.feature_importances_
feature_names = light_tuned_encoded.feature_name_

In [ ]:
sorted_idx = importances.argsort()[-10::]
feature_names_array = np.array(feature_names)
sorted_names = feature_names_array[sorted_idx]
greyish_white = '#D0D0D0'

fig = go.Figure(data=[
    go.Bar(y=sorted_names, 
           x=importances[sorted_idx], 
           orientation='h', 
           text=sorted_names,
#            textfont=dict(color=greyish_white),
           textposition='outside',
           marker={'color': importances[sorted_idx],
                   'colorscale': 'Viridis_r'})
])
fig.update_layout(
    title='Feature Importances',
#     title_font=dict(color=greyish_white),
    yaxis_title='Features',
#     yaxis_title_font=dict(color=greyish_white),  # Changes the y-axis title font color
    xaxis_title='Importance',
#     xaxis_title_font=dict(color=greyish_white),  # Changes the x-axis title font color
    yaxis_showticklabels=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        range=[0, 1100]  # Set your desired maximum value here
    )
)

# fig.write_html("plots/importances_top10.html")
fig.show()

### Summary of Model Progress

In [ ]:
new_data = {
    'Model': ['User Model'],
    'Version': ['M_E1'],
    'Dataset': ['df_user'],
    'MAE': [475.0987],
    'MSE': [652524.7350],
    'RMSE': [807.7900],
    'R2': [0.8952],
    'RMSLE': [0.3213],
    'MAPE': [0.2575]
}

new_metrics = pd.DataFrame(new_data)

In [ ]:
combined_metrics = combined_metrics.append(new_metrics, ignore_index=True)

### Pickles

In [ ]:
data_user = data
data_user_unseen = data_unseen

In [ ]:
with open('pickles/df_user.pkl', 'wb') as f:
    pickle.dump(df_user, f)
with open('pickles/data_user.pkl', 'wb') as f:
    pickle.dump(data_user, f)
with open('pickles/data_user_unseen.pkl', 'wb') as f:
    pickle.dump(data_user_unseen, f)
with open('pickles/combined_metrics.pkl', 'wb') as f:
    pickle.dump(combined_metrics, f)
with open('pickles/df_streamlit.pkl', 'wb') as f:
    pickle.dump(df_streamlit, f)
with open('pickles/df_plot_melted.pkl', 'wb') as f:
    pickle.dump(df_plot_melted, f)
with open('pickles/predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)
with open('pickles/predictions_user.pkl', 'wb') as f:
    pickle.dump(predictions_user, f)

In [ ]:
save_model(blended_user_tuned, 'models/blended_user_tuned')

In [ ]:
save_model(finalized_user_model, 'models/finalized_user_model')

### Pickling the Model for Deployment

In [ ]:
pickle.dump(finalized_user_model, open('models/finalized_user_model.pkl','wb'))

In [ ]:
from joblib import dump, load

dump(finalized_user_model, 'finalized_user_model.joblib')